In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
tr_features = np.load('train_voice_input.npy')
tr_labels = np.load('train_voice_label.npy')
ts_features = np.load('test_voice_input.npy')
ts_labels = np.load('test_voice_labels.npy')

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn import metrics

In [ ]:
train_x = tr_features
train_y = tr_labels
valid_x, test_x, valid_y, test_y = train_test_split(ts_features, ts_labels, test_size=0.5, random_state=42)

In [ ]:
frames = 41
bands = 60
num_channels = 2
num_labels = test_y.shape[1]

In [ ]:
f_size = 3
model = Sequential()

# Layer 1 - 24 filters with a receptive field of (f,f), i.e. W has the shape (24,1,f,f). 
# This is followed by (4,2) max-pooling over the last two dimensions and a ReLU activation function.
model.add(Convolution2D(256, f_size, f_size, border_mode='valid', input_shape=(bands, frames, num_channels)))
model.add(MaxPooling2D(pool_size=(4, 2)))
model.add(Activation('relu'))

model.add(Convolution2D(256, f_size, f_size, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(4, 2)))
model.add(Activation('relu'))

model.add(Convolution2D(256, f_size, f_size, border_mode='valid'))
model.add(Activation('relu'))

model.add(Flatten())

# Layer 4 - a fully connected NN layer of 64 hidden units, L2 penalty of 0.001
model.add(Dense(128, W_regularizer=l2(0.001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128, W_regularizer=l2(0.001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, W_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Activation('softmax'))
model.summary()
# create a SGD optimiser


In [ ]:



# a stopping function should the validation loss stop improving
earlystop = EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')

# compile and fit model, reduce epochs if you want a result faster
# the validation set is used to identify parameter settings (epoch) that achieves 
# the highest classification accuracy
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

model.fit(train_x, train_y, validation_data=(valid_x, valid_y), callbacks=[earlystop], batch_size=32, nb_epoch=50)

In [ ]:
# determine the classification accuracy
adam = Adam(lr=0.001)
model.save('voice_model.h5')
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

In [ ]:

score, accuracy = model.evaluate(test_x, test_y, batch_size=32)
print("\nAccuracy = {:.2f}".format(accuracy))
